Hyper-V 자동화 웹서비스 pay.py
===





적용 모듈 설명
------------
|모듈|설명|
|:---|:---|
|**flask**|플라스크 REST-API 설계를 위한 모듈|
|**pymongo**|python 내에서 mongoDB CRUD제어를 위한 모듈|
|request|결제 API로 tid를 받아오는데 사용한 모듈|
|json|json파일을 다루기 위한 모듈|


In [ ]:
from flask import Blueprint, redirect, session, render_template, request, jsonify, escape,url_for, make_response
import requests
import json
import pymongo
bp = Blueprint('bp', __name__)#블루프린트 연결

### 1. MongoDB연동

In [ ]:
client = pymongo.MongoClient("localhost",27017)
db = client.get_database("test")
admins = client.get_database("admin")
admin = admins.get_collection("admin")
user = db.get_collection("user")
Vm = db.get_collection("Vm")

### 2. point결제 페이지

In [ ]:
@bp.route('/point', methods=['GET','POST'])  
def point():
    user_id = session.get('login',None)#로그인 확인
    if user_id == None:
        return redirect(url_for("login"))
    if request.method == 'GET':
        return render_template('point.html',user_info = user.find_one({"user_id":user_id}), price = admin.find_one({"lable":"price"}))
    else:#admin이 설정한 할인율에 따라 결제 진행
        dc_col = admin.find_one({"lable":"DC"})
        DC_num = request.form.get('DC_num')
        bprice = float(request.form.get('pay_price'))
        dc = float(dc_col[DC_num])
        aprice = int(bprice*(1-0.01*dc))#할인 적용된 가격
        #이하 결제 준비단계(tid 받아오기)
        URL = 'https://kapi.kakao.com/v1/payment/ready'
        headers = {
            'Authorization': "KakaoAK " + "47505261b90a9bd874b965a27a6abc3e",
            "Content-type": "application/x-www-form-urlencoded;charset=utf-8",
        }
        params = {
            "cid": "TC0ONETIME", #가맹점 코드
            "partner_order_id": "1001",  #가맹점 주문코드
            "partner_user_id": user_id,  #가맹점 회원
            "item_name": "포인트)옵션:"+DC_num,  #상품명
            "quantity": 1, #상품 수량
            "total_amount": aprice, #상품총액 
            "tax_free_amount": 0,  #비과세 금액(그냥 0)
            "approval_url": "http://localhost:5000/kakaopay/success",#성공페이지
            "cancel_url": "http://localhost:5000/kakaopay/cancel",#취소페이지
            "fail_url": "http://localhost:5000/kakaopay/fail",#실패페이지
        }
        res = requests.post(URL, headers=headers, params=params)#서버 통신
        print(res)
        #request요청을 하면 세션이 초기화 되기 때문에 다시 세션에 저장해야함
        session["tid"] = res.json()['tid']#세션에 tid저장
        session['user_id'] = user_id#세션에 user_id저장
        return redirect(res.json()['next_redirect_pc_url'])

### 3. 결제 성공페이지


In [ ]:
@bp.route("/kakaopay/success", methods=['POST', 'GET'])
def sucess():
    #위와 같은 이유로 세션에 담아둔 tid,user_id를 변수에 저장
    tid = session.get("tid")
    user_id = session.get("user_id")
    
    URL = 'https://kapi.kakao.com/v1/payment/approve'
    headers = {
        "Authorization": "KakaoAK " + "47505261b90a9bd874b965a27a6abc3e",
        "Content-type": "application/x-www-form-urlencoded;charset=utf-8",
    }
    params = {
        "cid": "TC0ONETIME",  # 테스트용 코드
        "tid": tid,  # 결제 요청시 세션에 저장한 tid
        "partner_order_id": "1001",  # 주문번호
        "partner_user_id": user_id,  # 유저 아이디
        "pg_token": request.args.get("pg_token"),  # 쿼리 스트링으로 받은 pg토큰
    }
    res = requests.post(URL, headers=headers, params=params)
    print(res.text)
    print(res.json())
    print(res.json()['amount'])
    print(res.json()['amount']['total'])
    amount = res.json()['amount']['total']
    res = res.json()
    context = {
        'res': res,
        'amount': amount,
    }
    item = res['item_name'][7:]
    added_point= admin.find_one({"lable":"default_cash"})[item]
    session['user_id'] = user_id
    point = user.find_one({"user_id":user_id})["point"]
    point = point + added_point
    user.update_one({"user_id":user_id},{"$set":{"point":point}})
    return render_template('success.html', context=context, res=res)

### 4. 취소페이지

In [ ]:
@bp.route("/kakaopay/cancel", methods=['POST', 'GET'])
def cancel():
    URL = "https://kapi.kakao.com/v1/payment/order"
    headers = {
        "Authorization": "KakaoAK " + "47505261b90a9bd874b965a27a6abc3e",
        "Content-type": "application/x-www-form-urlencoded;charset=utf-8",
    }
    params = {
        "cid": "TC0ONETIME",  # 가맹점 코드
        "tid": session.get('tid'),  # 결제 고유 코드
    }
    res = requests.post(URL, headers=headers, params=params)
    print(res.text)
    amount = res.json()['cancel_available_amount']['total']
    context = {
        'res': res,
        'cancel_available_amount': amount,
    }
    
    if res.json()['status'] == "QUIT_PAYMENT":
        res = res.json()
        return render_template('cancel.html', params=params, res=res, context=context)


### 5. 결제 실패페이지

In [ ]:
@bp.route("/kakaopay/fail", methods=['POST', 'GET'])
def fail():

    return render_template('fail.html')